In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import json
import re

#Firstly, load the .json file to a DataFrame
with open('transaction-data-adhoc-analysis.json') as f:
    data = pd.read_json(f)

#Function to split multiple transactions by one buyer
def item_split_to_list(transaction_item):
    if ';' in transaction_item:
        item_list = transaction_item.split(';')
        return item_list
    else:
        return [transaction_item]
    
#Function to split the unique items into 'brand | item', and 'qty'
def brand_item(unique_item):
    item_detail_list = unique_item.split(',')
    return (item_detail_list[0]) + ' | ' + item_detail_list[1]
def qty(unique_item):
    item_detail_list = unique_item.split(',')
    return item_detail_list[2]

#Vectorize new function into the data DataFrame
data['transaction_items'] = data.apply(lambda x: item_split_to_list(x['transaction_items']), axis=1)

#Use .explode() function to expand the list into rows of the same index (will become a numpy array)
transaction_items_exploded = data['transaction_items'].explode()

#Create a new DF to store the numpy array
transaction_items_df = pd.DataFrame()

#Vectorize the 'brand | item' and 'qty' functions
vfunc_brand = np.vectorize(brand_item)
vfunc_qty = np.vectorize(qty)

#Dump the data into the 'transaction_items_df' using the vectorized functions
transaction_items_df['brand | item'] = vfunc_brand(transaction_items_exploded)
transaction_items_df['qty'] = vfunc_qty(transaction_items_exploded)

#Remove unwanted characters in the qty column (only gets the number part)
transaction_items_df['qty'] = transaction_items_df['qty'].str.replace(r'[^0-9]', '',regex=True)

#Convert 'qty' column into integers
transaction_items_df["qty"] = transaction_items_df["qty"].astype(str).astype(int)

#Create a new DataFrame to store unique store items w/o quantity getting in the way
inventory_df = pd.DataFrame(transaction_items_df['brand | item'].unique(), columns=['brand | item'])

#transaction_items_df

def total_qty_items(item_details, qty):
    total_qty = 0
    for i in range(len(inventory_df)):
        if inventory_df.iloc[i] == item_details:
            total_qty += qty
        else:
            total_qty +=0
        return total_qty
        
inventory_df['total qty']=transaction_items_df.apply(lambda x: total_qty_items(x['brand | item'], x['qty']),axis=1)
inventory_df

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().